In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Heading
After having gone through first dataset at (put link in future) this new dataset actually tells us the model. It also has robust features to make better ML models.

## Target variable
Price or variation of of price

The above function splits data and gives stratifying options. From here we import the data and bring it in

In [3]:
df = pd.read_csv('Cleaned_Laptop_data.csv')
df.columns

Index(['brand', 'model', 'processor_brand', 'processor_name',
       'processor_gnrtn', 'ram_gb', 'ram_type', 'ssd', 'hdd', 'os', 'os_bit',
       'graphic_card_gb', 'weight', 'display_size', 'warranty', 'Touchscreen',
       'msoffice', 'latest_price', 'old_price', 'discount', 'star_rating',
       'ratings', 'reviews'],
      dtype='object')

Adding lastest_price and old_price and dividng to get a working average
- working_average
# Now going into Column to column
We are assembling our questions for this dataset
I am starting at the bottom

In [4]:
df['reviews'].nunique()

152

The above finding is bizarre, I think this may be a mix of rating expressions

In [5]:
pd.DataFrame(df['reviews']).head(5)
#oops this NUMBER of reviews, not the reviews itself
#Can possibly clean up for clarity sake

,reviews
0,1947
1,108
2,4
3,18
4,15


# The reviews column
It is the number of reviews and maybe useful 

In [6]:
pd.DataFrame(df['ratings']).head(5)
#number of ratings

,ratings
0,15279
1,990
2,28
3,158
4,116


In [7]:
pd.DataFrame(df['star_rating']).head(5)
#THIS is what I was looking for a universal metric to for reviewing

,star_rating
0,3.8
1,4.3
2,3.9
3,4.4
4,4.2


In [8]:
df.star_rating.min()
#at position lowest star_rating review so like floor or min

0.0

In [9]:
df[df['star_rating'] == 0] 
#in the new findings there are a sizeable amount of 0s in the dataset for ratings which is worth interrogating

,brand,model,processor_brand,processor_name,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os,...,display_size,warranty,Touchscreen,msoffice,latest_price,old_price,discount,star_rating,ratings,reviews
42,Lenovo,Core,Intel,Core i5,11th,8,DDR4,512,0,Windows,...,15.6,1,No,No,52990,84890,37,0.0,0,0
43,acer,Aspire,Intel,Core i3,11th,8,DDR4,256,0,Windows,...,15.6,1,No,No,38990,52999,26,0.0,0,0
77,Lenovo,IdeaPad,Intel,Core i5,10th,8,DDR4,512,0,Windows,...,15.6,1,No,No,49990,79690,37,0.0,0,0
90,ASUS,VivoBook,Intel,Core i5,11th,8,DDR4,256,1024,Windows,...,15.6,1,No,No,52990,72990,27,0.0,0,0
99,ASUS,Zenbook,Intel,Core i7,11th,16,LPDDR4X,1024,0,Windows,...,14,1,Yes,Yes,104990,146990,28,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,Avita,Liber,Intel,Core i7,8th,8,DDR4,256,0,Windows,...,14,0,No,No,73990,79990,7,0.0,0,0
876,Avita,Liber,Intel,Core i7,8th,8,DDR4,256,0,Windows,...,14,0,No,No,73990,79990,7,0.0,0,0
877,Avita,Liber,Intel,Core i7,8th,8,DDR4,256,0,Windows,...,14,0,No,No,73990,79990,7,0.0,0,0
878,Avita,Liber,Intel,Core i7,8th,8,DDR4,256,0,Windows,...,14,0,No,No,73990,79990,7,0.0,0,0


In [10]:
df.columns

Index(['brand', 'model', 'processor_brand', 'processor_name',
       'processor_gnrtn', 'ram_gb', 'ram_type', 'ssd', 'hdd', 'os', 'os_bit',
       'graphic_card_gb', 'weight', 'display_size', 'warranty', 'Touchscreen',
       'msoffice', 'latest_price', 'old_price', 'discount', 'star_rating',
       'ratings', 'reviews'],
      dtype='object')

In [11]:
df['working_average'] = (df['latest_price'] + df['old_price']) / 2
df.head()

,brand,model,processor_brand,processor_name,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os,...,warranty,Touchscreen,msoffice,latest_price,old_price,discount,star_rating,ratings,reviews,working_average
0,ASUS,Celeron,Intel,Celeron Dual,Missing,4,DDR4,0,1024,Windows,...,1,No,No,23990,26990,11,3.8,15279,1947,25490.0
1,ASUS,VivoBook,Intel,Core i3,10th,8,DDR4,512,0,Windows,...,1,No,No,37990,50990,25,4.3,990,108,44490.0
2,ASUS,Vivobook,Intel,Core i3,10th,8,DDR4,0,1024,Windows,...,1,No,No,32890,46990,30,3.9,28,4,39940.0
3,HP,Core,Intel,Core i3,11th,8,DDR4,512,0,Windows,...,1,No,Yes,42990,57330,25,4.4,158,18,50160.0
4,HP,Core,Intel,Core i5,11th,8,DDR4,512,0,Windows,...,0,No,No,54990,70171,21,4.2,116,15,62580.5


In [12]:
df['working_average'].max()

361594.0

In [13]:
np.ceil(df['working_average'].sort_values(ascending= False))

377    361594.0
469    338490.0
229    319945.0
322    311090.0
865    307944.0
         ...   
803     19990.0
734     16995.0
887     16478.0
818     16459.0
776     10948.0
Name: working_average, Length: 896, dtype: float64

In [14]:
df.loc[469]

brand                  ASUS
model                   ROG
processor_brand       Intel
processor_name      Core i9
processor_gnrtn        12th
ram_gb                   32
ram_type               DDR5
ssd                    1024
hdd                       0
os                  Windows
os_bit                   64
graphic_card_gb           6
weight               Casual
display_size           15.6
warranty                  0
Touchscreen              No
msoffice                 No
latest_price         299990
old_price            376990
discount                 20
star_rating             0.0
ratings                   0
reviews                   0
working_average    338490.0
Name: 469, dtype: object

I think one of the leading variables of price will be branding. 
Since I am looking for most horsepower 
Graphics card for me cause I'm gamer
Companies would look for most CPU per dollar 
We can make multiple models changed per customer
One per customer base
We are starting with businesses
- The number of ratings is a corolation of how popular an item is


In [15]:
df.info()
#cleared up that there is 896 rather then a 1000 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 896 entries, 0 to 895
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   brand            896 non-null    object 
 1   model            896 non-null    object 
 2   processor_brand  896 non-null    object 
 3   processor_name   895 non-null    object 
 4   processor_gnrtn  896 non-null    object 
 5   ram_gb           871 non-null    object 
 6   ram_type         871 non-null    object 
 7   ssd              896 non-null    int64  
 8   hdd              896 non-null    int64  
 9   os               896 non-null    object 
 10  os_bit           896 non-null    int64  
 11  graphic_card_gb  896 non-null    int64  
 12  weight           896 non-null    object 
 13  display_size     896 non-null    object 
 14  warranty         896 non-null    int64  
 15  Touchscreen      896 non-null    object 
 16  msoffice         896 non-null    object 
 17  latest_price    

In [17]:
slab = df.head().T
slab
#slab will be the shorthand to bring a small slice of the table into the mix

,0,1,2,3,4
brand,ASUS,ASUS,ASUS,HP,HP
model,Celeron,VivoBook,Vivobook,Core,Core
processor_brand,Intel,Intel,Intel,Intel,Intel
processor_name,Celeron Dual,Core i3,Core i3,Core i3,Core i5
processor_gnrtn,Missing,10th,10th,11th,11th
ram_gb,4,8,8,8,8
ram_type,DDR4,DDR4,DDR4,DDR4,DDR4
ssd,0,512,0,512,512
hdd,1024,0,1024,0,0
os,Windows,Windows,Windows,Windows,Windows


In [23]:
df[df['processor_gnrtn'] == "12th"]
# the current generational model is 13th this informs us that the dataset is a little olf
# on further investigation there are much more older models then new

,brand,model,processor_brand,processor_name,processor_gnrtn,ram_gb,ram_type,ssd,hdd,os,...,warranty,Touchscreen,msoffice,latest_price,old_price,discount,star_rating,ratings,reviews,working_average
466,MSI,Stealth,Intel,Core i7,12th,32,DDR5,1024,0,Windows,...,0,No,No,279990,313990,10,0.0,0,0,296990.0
469,ASUS,ROG,Intel,Core i9,12th,32,DDR5,1024,0,Windows,...,0,No,No,299990,376990,20,0.0,0,0,338490.0
536,MSI,Core,Intel,Core i7,12th,16,DDR4,1024,0,Windows,...,0,No,No,139990,156990,10,4.5,4,2,148490.0


The 1/3rd gap of processor_gnrtn is "Missing", this will create a bias towards the proceeser generation named missing. 

From outside discussion we have narrowed our audience to companies getting the best bang for buck when buying in large quantites.
- This may not include discounts

In [24]:
df.isnull().sum()

brand               0
model               0
processor_brand     0
processor_name      1
processor_gnrtn     0
ram_gb             25
ram_type           25
ssd                 0
hdd                 0
os                  0
os_bit              0
graphic_card_gb     0
weight              0
display_size        0
warranty            0
Touchscreen         0
msoffice            0
latest_price        0
old_price           0
discount            0
star_rating         0
ratings             0
reviews             0
working_average     0
dtype: int64

In [25]:
df.dtypes

brand               object
model               object
processor_brand     object
processor_name      object
processor_gnrtn     object
ram_gb              object
ram_type            object
ssd                  int64
hdd                  int64
os                  object
os_bit               int64
graphic_card_gb      int64
weight              object
display_size        object
warranty             int64
Touchscreen         object
msoffice            object
latest_price         int64
old_price            int64
discount             int64
star_rating        float64
ratings              int64
reviews              int64
working_average    float64
dtype: object

Since we have a target variable we should be able to do Tests
- Like ANOVA or other sorts of equations to measure weather or not they have pull on price
"We should do statistical tests"
- Our goal is to make a tier list of useful corporate large purchase laptops (in a ideal world we can pull 1 specific one)
- This would roughly be a clustering to make groups of value statements
- for our audience we can even give specific laptops for what metrics the company is looking for

"A good starting point is to cluster. Then use those clusters to start our binning. These clusters would be grouping different laptops. Then we can recommend different groups based off of the audience"
- possible pivot in audience to recommend for schools instead to get laptops

Currently in this exploration phase we haven't quite found the best indicators of oull on price (our primary goal to work around). <br>
I think the next thing to do here is some outside research into what schools needs the most of the function and frankly I think it is speed, the laptops are not going to need a large amount of hard drive space as much of the information can be stored "in the clouds".
- This would then almost certainly cut out a need for MACS
But now furthering the questions coming into this dataset we are with another question?
- What <u>kind</u> of schools will we be shooting this towards.....
underprivleged ones?